# Fatigue Detection Machine Learning Model

## Package Imports

In [1]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.io import read_video
from torchvision.transforms import ToTensor

## Data Import

In [2]:
# Custom Dataset Class
class VideoDataset(Dataset):
    def __init__(self, data_dir, test=False):
        self.data_dir = data_dir
        self.samples = []
        self.test = test

        for subject_folder in os.listdir(data_dir):
            for case_folder in os.listdir(os.path.join(data_dir, subject_folder)):
                for file in os.listdir(os.path.join(data_dir, subject_folder, case_folder)):
                    if file.endswith(('.avi', '.mp4')):
                        file_path = os.path.join(
                            data_dir, subject_folder, case_folder, file)
                        if test:
                            self.samples.append({
                                'video_path': file_path,
                                'drowsy_label': file_path[:-4] + 'ing_drowsiness.txt',
                            })
                        else:
                            self.samples.append({
                                'video_path': file_path,
                                'drowsy_label': file_path[:-4] + '_drowsiness.txt',
                                'eye_label': file_path[:-4] + '_eye.txt',
                                'head_label': file_path[:-4] + '_head.txt',
                                'mouth_label': file_path[:-4] + '_mouth.txt'
                            })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        video_data = read_video(sample['video_path'])
        return video_data

In [3]:
training_data = VideoDataset("./Data/Training_Evaluation_Dataset/Training_Dataset")
evaluation_data = VideoDataset("./Data/Training_Evaluation_Dataset/Evaluation_Dataset")
testing_data = VideoDataset("./Data/Testing_Dataset/", True)

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

./Data/Testing_Dataset/003\glasses\003_glasses_mix.mp4


c:\Users\zein_\anaconda3\envs\Dissertation\lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


KeyboardInterrupt: 

## Neural Network

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312170  [   64/60000]
loss: 2.293956  [ 6464/60000]
loss: 2.273635  [12864/60000]
loss: 2.263413  [19264/60000]
loss: 2.256236  [25664/60000]
loss: 2.227988  [32064/60000]
loss: 2.230313  [38464/60000]
loss: 2.205057  [44864/60000]
loss: 2.204941  [51264/60000]
loss: 2.165660  [57664/60000]
Test Error: 
 Accuracy: 40.9%, Avg loss: 2.164699 

Epoch 2
-------------------------------
loss: 2.178445  [   64/60000]
loss: 2.169152  [ 6464/60000]
loss: 2.111590  [12864/60000]
loss: 2.126540  [19264/60000]
loss: 2.089863  [25664/60000]
loss: 2.025366  [32064/60000]
loss: 2.055251  [38464/60000]
loss: 1.980800  [44864/60000]
loss: 1.988140  [51264/60000]
loss: 1.921827  [57664/60000]
Test Error: 
 Accuracy: 58.3%, Avg loss: 1.916599 

Epoch 3
-------------------------------
loss: 1.946239  [   64/60000]
loss: 1.924348  [ 6464/60000]
loss: 1.804683  [12864/60000]
loss: 1.846665  [19264/60000]
loss: 1.753213  [25664/60000]
loss: 1.690940  [32064/600